# 预处理assets，整理出后续要用的数据

In [1]:
from dotenv import load_dotenv
import os
os.environ['DEBUG'] = '1'
os.environ['SHOW_LLM_INPUT_MSG'] = '1'

# 加载 .env 文件
load_dotenv()

True

In [2]:
import pandas as pd
import json
import llms
from src.utils import show

In [3]:
# Preprocess the competition questions here
root_dir = os.getcwd()
question_data_path = root_dir + '/assets/question.json'
df1 = pd.read_excel(root_dir + '/assets/数据字典.xlsx', sheet_name='库表关系')
df2 = pd.read_excel(root_dir + '/assets/数据字典.xlsx', sheet_name='表字段信息')
file_path = root_dir + '/assets/all_tables_schema.txt'

In [4]:
# 假设 df1 的列名为 '库名英文', '库名中文', '表英文', '表中文', '表描述'
db_table = df1.groupby('库名英文').apply(lambda x: {
    '库名中文': x['库名中文'].iloc[0],
    '表': x[['表英文', '表中文', '表描述']].apply(lambda y: y.str.lower() if y.name == '表英文' else y).to_dict('records')
}).to_dict()
db_table = {k.lower(): v for k, v in db_table.items()}

# 打印结果以验证
with open(root_dir + '/assets/db_table.json', 'w', encoding='utf-8') as json_file:
    json.dump(db_table, json_file, ensure_ascii=False, indent=2)

/tmp/ipykernel_846877/3268220370.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  db_table = df1.groupby('库名英文').apply(lambda x: {


In [5]:
# 假设 df2 的列名为 'table_name', 'column_name', 'column_description', '注释'
table_column = df2.groupby('table_name').apply(lambda x: [
    {
        'column': row['column_name'],
        'desc': f"{row['column_description']}" + (f"；{row['注释']}" if pd.notna(row['注释']) and row['注释'] else "").strip()
    }
    for _, row in x.iterrows() if pd.notna(row['column_name']) and row['column_name'] not in ['JSID', 'UpdateTime', 'InsertTime', 'ID', 'XGRQ']
]).to_dict()
table_column = {k.lower(): v for k, v in table_column.items()}

# 打印结果以验证
with open(root_dir + '/assets/table_column.json', 'w', encoding='utf-8') as json_file:
    json.dump(table_column, json_file, ensure_ascii=False, indent=2)

/tmp/ipykernel_846877/1464820833.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  table_column = df2.groupby('table_name').apply(lambda x: [


In [7]:
for db_name, db in db_table.items():
    for table in db["表"]:
        table_name = table["表英文"]
        print(f"{db_name}.{table_name}")
        column_list = []
        for column in table_column[table_name]:
            column_name = column["column"]
            column_simple_desc = str(column["desc"]).split("；")[0]
            column_list.append(f"{column_simple_desc}({column_name})")
        all_cols = ",".join(column_list)
        table["all_cols"] = all_cols
        cols_summary, token, ok = llms.llm_glm_4_plus.generate_response(
            system='''你善于对数据表的字段信息进行总结，把同类信息归类，比如"联系人电话、联系人传真"等总结为"联系方式如电话、传真等。
        输出一段文字，不换行。"''',
            messages=[
                {
                    "role": "user",
                    "content": f"下面是一个数据表的所有表字段，请帮我为这个数据表写一段介绍，把字段信息压缩进去：\n{all_cols}"
                }
            ],
            stream=False,
        )
        table["cols_summary"] = cols_summary
        if not ok:
            print(f"err: {db_name}.{table_name}")

astockbasicinfodb.lc_stockarchives
该数据表涵盖公司基本信息、联系方式、注册及办公地址、证券信息、高管及顾问信息等。包括公司代码、国别、董事会秘书及其联系方式、证券事务代表及其联系方式、授权代表、多种联系方式（电话、传真、邮箱）、注册及办公地址及邮编、公司网址、信息披露渠道、成立日期、法人代表、总经理、法律顾问、会计师事务所、证监会行业分类、经营范围、A股/B股/H股/CDR证券信息、公司简介、营业执照注册号、地区代码、特殊标识（盈利、表决权、控制架构、红筹）及所属区县等字段。
astockbasicinfodb.lc_namechange
该数据表涵盖公司基本信息及决议公告情况，包括公司代码、信息发布日期及来源，股东大会决议公告日期及否决情况，公司全称及简称的中英文变更日期与具体名称。
astockbasicinfodb.lc_business
该数据表涵盖公司基本信息及业务范围，包括公司代码、信息发布日期及来源、股东大会决议公告日期、否决情况、主营及兼营范围、主要业务与产品名称、行业相关代码及类别、涉足行业以及简称变更原因等关键信息。
astockeventsdb.lc_warrant
该数据表涵盖公司事件相关信息，包括公司代码、信息发布日期及来源、公告类型与披露方式、事件内容描述及进展状态、事件主体与进程、行为方式及货币单位、主体与对象名称及关联关系、协议签署及终止情况、担保原因及金额、担保企业与资产、反担保信息、借贷银行及期限、担保期限及解除详情，以及违规与逾期情况等。
astockeventsdb.lc_credit
该数据表涵盖公司借贷及相关信息披露的详细信息，包括公司代码、信息发布日期及来源、公告类型与披露方式、事件内容与行为描述、事件主体及进程、交易对象及关联关系、借贷条件与金额、还款情况、利率与期限、担保信息及日期等关键字段，全面记录借贷事件的各个环节与相关方信息。
astockeventsdb.lc_suitarbitration
该数据表详细记录了公司相关事件的全面信息，包括基础标识（ID、公司代码）、信息发布情况（首次及最新发布日期、来源、公告类型、披露方式）、事件详情（内容、行为描述、最新进展、主体及进程、行为方式）、财务相关（货币单位、涉及金额）、关联方信息（事件主体及交易对象名称、编号、与上市公司关联关

In [8]:
with open(root_dir + '/assets/db_table.json', 'w', encoding='utf-8') as json_file:
    json.dump(db_table, json_file, ensure_ascii=False, indent=2)

In [9]:
db_info = []
for db_name, db in db_table.items():
    print(db_name)
    db_json = json.dumps(db, ensure_ascii=False)
    db_summary, token, ok = llms.llm_glm_4_plus.generate_response(
    system='''你善于对数据库的表信息进行总结，根据它包含的数据表和字段信息，描述这个数据库，如"本库名为xxx，记录了xxx；涵盖了xxx；方便用户xxx"。
输出一段文字，不换行。"''',
    messages=[
        {
            "role": "user",
            "content": f"下面是一个数据库的所有表和字段信息，请帮我为这个数据库写一段介绍，把表和字段信息压缩进去：\n{db}"
        }
    ],
    stream=False,
)
    db_info.append({
        "db_name": db_name,
        "db_desc": db_summary,
    })

astockbasicinfodb
本库名为“上市公司基本资料”，记录了上市公司的核心信息及变更情况；涵盖了公司概况、名称更改状况、经营范围与行业变更三大方面。具体包括公司基本信息、联系方式、注册及办公地址、证券信息、高管及顾问信息、名称变更历史、业务范围及行业分类等详细数据。方便用户全面了解上市公司的基础资料、历史变迁及业务发展，为投资决策、研究分析提供有力支持。
astockeventsdb
本库名为“上市公司公告资讯/重大事项”，记录了上市公司公告中披露的担保、借贷、诉讼仲裁、委托理财、资产重组、重大经营合同及投资者关系活动等重大事项；涵盖了公司代码、信息发布日期、事件内容、主体与交易对象信息、财务数据、协议签署情况、进展状态、关联关系、担保与借贷细节、诉讼仲裁状态、委托理财条款、资产重组财务指标、合同标的与期限、调研活动详情等字段；方便用户全面了解上市公司重大事项的详细信息，进行数据分析和决策支持。
astockfinancedb
本库名为上市公司财务指标/财务报表/融资与分红，记录了A股增发、A股配股、公司分红、资金投向说明、资产负债表_新会计准则、利润分配表_新会计准则、现金流量表_新会计准则、公司研发投入与产出、公司主营业务构成、公司经营情况述评、公司历年审计意见等数据，涵盖了证券发行全流程信息、配股相关信息、分红情况、募资及收购兼并相关信息、公司财务报表详细信息、公司财务信息、公司财务及现金流量详细信息、公司研发投入及相关人员信息、公司经营信息、公司经营状况及相关信息披露时间、公司审计相关信息等，方便用户全面了解上市公司的财务状况、经营情况、审计意见等信息。
astockindustrydb
本库名为“上市公司行业板块”，记录了上市公司在多种行业划分标准下的所属行业情况及其变更历史，涵盖了公司行业划分表、公司行业变更表、行业估值指标、行业财务指标表、概念所属公司表及概念板块常量表；方便用户进行公司行业分类查询、行业变更追溯、多维度财务指标分析及概念板块信息检索，支持投资者数据回测、行业估值计算及市场热点概念分析。
astockmarketquotesdb
本库名为上市公司股票行情，记录了境内股票交易资金流向指标、境内股票成交量技术指标、股票技术形态表、日行情表、股票行情表现(新)、停牌复牌表；涵盖了证券交易的多维度数据，包括证券内部编码、交易日期、成

In [10]:
with open(root_dir + '/assets/db_info.json', 'w', encoding='utf-8') as json_file:
    json.dump(db_info, json_file, ensure_ascii=False, indent=2)